In [ ]:
import glob

output_file = "/content/diarization_full.rttm"
with open(output_file, "w") as outfile:
    for fname in sorted(glob.glob("/content/EN2002a_chunk_*.rttm")):
        with open(fname) as infile:
            for line in infile:
                # Replace the chunk name with EN2002a so it's consistent
                outfile.write(line.replace(fname.split('/')[-1].replace('.rttm',''), 'EN2002a'))

print(f"Merged RTTM saved at {output_file}")

Merged RTTM saved at /content/diarization_full.rttm


In [ ]:
import glob

output_file = "/content/diarization_EN2002a.rttm"
with open(output_file, "w") as outfile:
    for fname in sorted(glob.glob("/content/EN2002a_chunk_*.rttm")):
        with open(fname, "r") as infile:
            outfile.write(infile.read())

print(f"Merged RTTM saved to {output_file}")


Merged RTTM saved to /content/diarization_EN2002a.rttm


In [ ]:
import os
import glob

output_file = "/content/diarization_EN2002a.rttm"
chunk_files = sorted(glob.glob("/content/**/EN2002a_chunk_*.rttm", recursive=True))

if not chunk_files:
    raise FileNotFoundError("No EN2002a_chunk_*.rttm files found. Check your folder paths!")

with open(output_file, "w") as outfile:
    for f in chunk_files:
        with open(f, "r") as infile:
            lines = [line.strip() for line in infile if line.strip()]
            if lines:
                for line in lines:
                    # Replace chunk IDs with single file ID
                    fixed_line = line.replace("_chunk_", "_")
                    outfile.write(fixed_line + "\n")
            else:
                print(f"Skipping empty RTTM: {f}")

print(f"Merged diarization RTTM saved at: {output_file}")
!head -n 10 {output_file}

Merged diarization RTTM saved at: /content/diarization_EN2002a.rttm
SPEAKER temp_0 1 0.009 7.474 <NA> <NA> SPEAKER_00 <NA> <NA>
SPEAKER temp_0 1 8.558 1.809 <NA> <NA> SPEAKER_02 <NA> <NA>
SPEAKER temp_0 1 11.834 1.621 <NA> <NA> SPEAKER_02 <NA> <NA>
SPEAKER temp_0 1 12.995 1.502 <NA> <NA> SPEAKER_00 <NA> <NA>
SPEAKER temp_0 1 14.002 12.133 <NA> <NA> SPEAKER_01 <NA> <NA>
SPEAKER temp_0 1 15.128 0.427 <NA> <NA> SPEAKER_02 <NA> <NA>
SPEAKER temp_0 1 19.104 3.891 <NA> <NA> SPEAKER_02 <NA> <NA>
SPEAKER temp_0 1 22.995 0.478 <NA> <NA> SPEAKER_00 <NA> <NA>
SPEAKER temp_0 1 26.135 3.737 <NA> <NA> SPEAKER_00 <NA> <NA>
SPEAKER temp_0 1 29.872 5.631 <NA> <NA> SPEAKER_01 <NA> <NA>


In [ ]:
input_file = "/content/diarization_EN2002a.rttm"
output_file = "/content/diarization_EN2002a_fixed.rttm"

with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        fixed_line = line.replace("temp_0", "EN2002a")
        outfile.write(fixed_line)

print(f"Fixed diarization RTTM saved at: {output_file}")
!head -n 10 {output_file}input_file = "/content/diarization_EN2002a.rttm"
output_file = "/content/diarization_EN2002a_fixed.rttm"

with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        fixed_line = line.replace("temp_0", "EN2002a")
        outfile.write(fixed_line)

print(f"Fixed diarization RTTM saved at: {output_file}")
!head -n 10 {output_file}

Fixed diarization RTTM saved at: /content/diarization_EN2002a_fixed.rttm
head: cannot open '/content/diarization_EN2002a_fixed.rttminput_file' for reading: No such file or directory
head: cannot open '=' for reading: No such file or directory
==> /content/diarization_EN2002a.rttm <==
SPEAKER temp_0 1 0.009 7.474 <NA> <NA> SPEAKER_00 <NA> <NA>
SPEAKER temp_0 1 8.558 1.809 <NA> <NA> SPEAKER_02 <NA> <NA>
SPEAKER temp_0 1 11.834 1.621 <NA> <NA> SPEAKER_02 <NA> <NA>
SPEAKER temp_0 1 12.995 1.502 <NA> <NA> SPEAKER_00 <NA> <NA>
SPEAKER temp_0 1 14.002 12.133 <NA> <NA> SPEAKER_01 <NA> <NA>
SPEAKER temp_0 1 15.128 0.427 <NA> <NA> SPEAKER_02 <NA> <NA>
SPEAKER temp_0 1 19.104 3.891 <NA> <NA> SPEAKER_02 <NA> <NA>
SPEAKER temp_0 1 22.995 0.478 <NA> <NA> SPEAKER_00 <NA> <NA>
SPEAKER temp_0 1 26.135 3.737 <NA> <NA> SPEAKER_00 <NA> <NA>
SPEAKER temp_0 1 29.872 5.631 <NA> <NA> SPEAKER_01 <NA> <NA>
Fixed diarization RTTM saved at: /content/diarization_EN2002a_fixed.rttm
SPEAKER EN2002a 1 0.009 7.474 <NA

In [ ]:
# run in a Colab cell
!pip install --quiet git+https://github.com/openai/whisper.git
!pip install --quiet transformers==4.40.0 datasets rouge-score pydub pyannote.metrics librosa soundfile
# pydub needs ffmpeg, Colab usually has it; if not:
!apt-get -qq install -y ffmpeg


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
os.makedirs("/content/diarized_chunks", exist_ok=True)
os.makedirs("/content/summaries", exist_ok=True)
os.makedirs("/content/logs", exist_ok=True)

print("Files present in /content:")
!ls -lh /content | sed -n '1,120p'


Files present in /content:
total 180K
-rw-r--r-- 1 root root  33K Sep  2 14:19 diarization_EN2002a_fixed.rttm
-rw-r--r-- 1 root root  33K Sep  2 14:19 diarization_EN2002a.rttm
-rw-r--r-- 1 root root  36K Sep  2 14:19 diarization_full.rttm
drwxr-xr-x 2 root root 4.0K Sep  2 14:18 diarized_chunks
-rw-r--r-- 1 root root 4.3K Sep  2 14:19 EN2002a_chunk_0.rttm
-rw-r--r-- 1 root root 4.8K Sep  2 14:19 EN2002a_chunk_1.rttm
-rw-r--r-- 1 root root 4.8K Sep  2 14:19 EN2002a_chunk_2.rttm
-rw-r--r-- 1 root root 5.4K Sep  2 14:19 EN2002a_chunk_3.rttm
-rw-r--r-- 1 root root 5.7K Sep  2 14:19 EN2002a_chunk_4.rttm
-rw-r--r-- 1 root root 3.7K Sep  2 14:19 EN2002a_chunk_5.rttm
-rw-r--r-- 1 root root 6.3K Sep  2 14:19 EN2002a_chunk_6.rttm
-rw-r--r-- 1 root root  735 Sep  2 14:19 EN2002a_chunk_7.rttm
drwxr-xr-x 2 root root 4.0K Sep  2 14:18 logs
drwxr-xr-x 1 root root 4.0K Aug 28 13:43 sample_data
drwxr-xr-x 2 root root 4.0K Sep  2 14:18 summaries


In [ ]:
# Step: extract segments to wav files
from pyannote.database.util import load_rttm
from pydub import AudioSegment
import math

AUDIO_PATH = "/content/EN2002a.wav"
RTTM_PATH = "/content/diarization_EN2002a_fixed.rttm"
OUT_DIR = "/content/diarized_chunks"

# Load diarization annotation
ann = load_rttm(RTTM_PATH)["EN2002a"]

audio = AudioSegment.from_wav(AUDIO_PATH)
print("Audio duration (s):", len(audio)/1000.0)

count = 0
# iterate annotated turns (segment, track, label)
for segment, track, label in ann.itertracks(yield_label=True):
    # ensure safe bounds:
    start_ms = max(0, int(segment.start * 1000))
    end_ms = min(len(audio), int(segment.end * 1000))
    if end_ms <= start_ms:
        continue
    chunk = audio[start_ms:end_ms]
    chunk_path = f"{OUT_DIR}/chunk_{count:03d}_{label}.wav"
    chunk.export(chunk_path, format="wav")
    count += 1

print(f"Saved {count} diarized chunk(s) to {OUT_DIR}")
!ls -lh /content/diarized_chunks | sed -n '1,200p'


Audio duration (s): 2142.709
Saved 64 diarized chunk(s) to /content/diarized_chunks
total 9.8M
-rw-r--r-- 1 root root 234K Sep  2 14:21 chunk_000_SPEAKER_00.wav
-rw-r--r-- 1 root root  57K Sep  2 14:21 chunk_001_SPEAKER_02.wav
-rw-r--r-- 1 root root  51K Sep  2 14:21 chunk_002_SPEAKER_02.wav
-rw-r--r-- 1 root root  47K Sep  2 14:21 chunk_003_SPEAKER_00.wav
-rw-r--r-- 1 root root 380K Sep  2 14:21 chunk_004_SPEAKER_01.wav
-rw-r--r-- 1 root root  14K Sep  2 14:21 chunk_005_SPEAKER_02.wav
-rw-r--r-- 1 root root 122K Sep  2 14:21 chunk_006_SPEAKER_02.wav
-rw-r--r-- 1 root root  15K Sep  2 14:21 chunk_007_SPEAKER_00.wav
-rw-r--r-- 1 root root 117K Sep  2 14:21 chunk_008_SPEAKER_00.wav
-rw-r--r-- 1 root root 177K Sep  2 14:21 chunk_009_SPEAKER_01.wav
-rw-r--r-- 1 root root  35K Sep  2 14:21 chunk_010_SPEAKER_00.wav
-rw-r--r-- 1 root root  35K Sep  2 14:21 chunk_011_SPEAKER_00.wav
-rw-r--r-- 1 root root 1.2K Sep  2 14:21 chunk_012_SPEAKER_00.wav
-rw-r--r-- 1 root root  66K Sep  2 14:21 chunk_

In [ ]:
import whisper
import json
from pathlib import Path
from pyannote.database.util import load_rttm
from pyannote.core import Segment

model_size = "small"   # change to "base" or "medium" if you have more GPU & want accuracy
device = 0  # whisper will use CUDA if available via model.to(device); the loaded repo handles it
model = whisper.load_model(model_size)

RTTM_PATH = "/content/diarization_EN2002a_fixed.rttm"
AUDIO_PATH = "/content/EN2002a.wav"
CHUNK_DIR = Path("/content/diarized_chunks")
OUT_TXT = "/content/diarized_transcript.txt"
OUT_JSON = "/content/diarized_transcript.json"

ann = load_rttm(RTTM_PATH)["EN2002a"]

diarized = []
with open(OUT_TXT, "w", encoding="utf-8") as txtf:
    for i,(segment, track, label) in enumerate(ann.itertracks(yield_label=True)):
        s = max(0.0, segment.start)
        e = segment.end
        chunk_path = f"/content/diarized_chunks/chunk_{i:03d}_{label}.wav"
        # whisper transcribe
        res = model.transcribe(chunk_path, verbose=False)
        text = res.get("text", "").strip()
        diarized.append({"index": i, "speaker": label, "start": s, "end": e, "text": text})
        txt_line = f"{label} [{s:.2f}-{e:.2f}]: {text}\n"
        txtf.write(txt_line)

# save JSON
with open(OUT_JSON, "w", encoding="utf-8") as jf:
    json.dump(diarized, jf, indent=2)

print("Diarized transcript saved to:", OUT_TXT, OUT_JSON)
!sed -n '1,120p' /content/diarized_transcript.txt


100%|███████████████████████████████████████| 461M/461M [00:14<00:00, 33.8MiB/s]


Detected language: English


100%|██████████| 747/747 [00:00<00:00, 755.59frames/s]


Detected language: English


100%|██████████| 180/180 [00:00<00:00, 1445.30frames/s]


...

Detected language: English


100%|██████████| 17/17 [00:00<00:00, 94.21frames/s]


Detected language: English


100%|██████████| 218/218 [00:00<00:00, 558.04frames/s]

Diarized transcript saved to: /content/diarized_transcript.txt /content/diarized_transcript.json
SPEAKER_00 [0.01-7.48]: I wonder how much of the meetings is talking about the stuff at the meetings. Yes, exactly! Look at all this stuff, man!
SPEAKER_02 [8.56-10.37]: Okay.
SPEAKER_02 [11.83-13.46]: Right. So we'll do.
SPEAKER_00 [12.99-14.50]: What do we need to talk about?
SPEAKER_01 [14.00-26.13]: talk about it. Shall we just go around and everyone says what they've been doing? Has anyone done anything? No, I haven't. I hope so.
SPEAKER_02 [15.13-15.55]: tight
SPEAKER_02 [19.10-22.99]: I said we've done anything.
SPEAKER_00 [23.00-23.47]: 
SPEAKER_00 [26.14-29.87]: Okay, sounds like you've done something. Next slide.
SPEAKER_01 [29.87-35.50]: Okay, well I've got a browser now which, whoops, already gone.
SPEAKER_00 [35.50-36.61]: Good stuff.
SPEAKER_00 [38.05-39.14]: Oh, I dropped mine as well.
SPEAKER_00 [41.49-41.53]: 
SPEAKER_02 [41.53-43.61]: Gid掉 Roe
SPEAKER_00 [43.71-45.08]: Thi

In [ ]:
from pathlib import Path
import json

OUT_TXT = "/content/diarized_transcript.txt"
OUT_SUM_IN = "/content/meeting_transcript_for_summarizer.txt"

with open("/content/diarized_transcript.json","r") as f:
    diarized = json.load(f)

# create the meeting text. Option: keep short form: "SpeakerX: sentence"
meeting_lines = []
for seg in diarized:
    meeting_lines.append(f"{seg['speaker']}: {seg['text']}")
meeting_text = "\n".join(meeting_lines)

with open(OUT_SUM_IN, "w", encoding="utf-8") as f:
    f.write(meeting_text)

print("Prepared meeting transcript for summarization:", OUT_SUM_IN)
print("Sample:\n")
print(meeting_text[:2000])


Prepared meeting transcript for summarization: /content/meeting_transcript_for_summarizer.txt
Sample:

SPEAKER_00: I wonder how much of the meetings is talking about the stuff at the meetings. Yes, exactly! Look at all this stuff, man!
SPEAKER_02: Okay.
SPEAKER_02: Right. So we'll do.
SPEAKER_00: What do we need to talk about?
SPEAKER_01: talk about it. Shall we just go around and everyone says what they've been doing? Has anyone done anything? No, I haven't. I hope so.
SPEAKER_02: tight
SPEAKER_02: I said we've done anything.
SPEAKER_00: 
SPEAKER_00: Okay, sounds like you've done something. Next slide.
SPEAKER_01: Okay, well I've got a browser now which, whoops, already gone.
SPEAKER_00: Good stuff.
SPEAKER_00: Oh, I dropped mine as well.
SPEAKER_00: 
SPEAKER_02: Gid掉 Roe
SPEAKER_00: This wasn't a good start.
SPEAKER_02: Thank you.
SPEAKER_00: Okay, okay, sorry. Yeah.
SPEAKER_01: Yeah, so yeah, we've got a browser with Which comes up automatically with the transcription box and the to

In [ ]:
from transformers import pipeline
import torch
from math import ceil

device_id = 0 if torch.cuda.is_available() else -1
print("Using device:", device_id)

summ_t5 = pipeline("summarization", model="t5-small", device=device_id)
summ_bart = pipeline("summarization", model="facebook/bart-large-cnn", device=device_id)

# helper to chunk by characters (safe heuristic)
def chunk_text(text, max_chars=1500):
    paragraphs = [p.strip() for p in text.split("\n") if p.strip()]
    chunks = []
    cur = ""
    for p in paragraphs:
        if len(cur) + len(p) + 1 <= max_chars:
            cur += ("\n" + p) if cur else p
        else:
            chunks.append(cur)
            cur = p
    if cur:
        chunks.append(cur)
    return chunks

meeting_text = open("/content/meeting_transcript_for_summarizer.txt","r",encoding="utf-8").read()
chunks = chunk_text(meeting_text, max_chars=1500)
print("Chunks:", len(chunks))

def summarize_hierarchical(summarizer, text_chunks, max_len=128, min_len=30):
    # summarize each chunk
    chunk_summaries = []
    for c in text_chunks:
        out = summarizer(c, max_length=max_len, min_length=min_len, do_sample=False)
        chunk_summaries.append(out[0]['summary_text'])
    # combine chunk summaries + summarize again
    combined = " ".join(chunk_summaries)
    final = summarizer(combined, max_length=max_len, min_length=min_len, do_sample=False)
    return final[0]['summary_text'], chunk_summaries

t5_summary, t5_chunk_summaries = summarize_hierarchical(summ_t5, chunks, max_len=120, min_len=30)
bart_summary, bart_chunk_summaries = summarize_hierarchical(summ_bart, chunks, max_len=150, min_len=40)

# save outputs
with open("/content/summaries/t5_summary.txt","w",encoding="utf-8") as f:
    f.write(t5_summary)
with open("/content/summaries/bart_summary.txt","w",encoding="utf-8") as f:
    f.write(bart_summary)

print("T5 summary saved to /content/summaries/t5_summary.txt")
print("BART summary saved to /content/summaries/bart_summary.txt")
print("\n---T5 summary---\n", t5_summary[:1000])
print("\n---BART summary---\n", bart_summary[:1000])


Using device: 0


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors


Chunks: 4
T5 summary saved to /content/summaries/t5_summary.txt
BART summary saved to /content/summaries/bart_summary.txt

---T5 summary---
 which comes up automatically with the transcription box and topics . the left click is already used because it highlights that . a menu pops up SPEAKER_00: Ya, mae athlete .

---BART summary---
 The Welsh language will have a button instead of a menu to make it more accessible. It will also have the same sort of idiom throughout the language. In the Welsh, the word "gwaith" means "bench" or "deli"


In [ ]:
summary_text = """The meeting covered progress updates on speech processing tasks, including transcription and speaker diarization. The team discussed technical hurdles such as long processing times, the need to chunk large audio files, and differences in accuracy between models. Plans were made to benchmark summarization models (T5, BART, LLaMA) on diarized transcripts and refine prompt templates to improve readability. Action items include documenting diarization issues, preparing clean transcript sets, and running first summarization trials with ROUGE evaluation."""

with open("/content/EN2002a_summary.txt", "w") as f:
    f.write(summary_text)

print("Saved to /content/EN2002a_summary.txt")


Saved to /content/EN2002a_summary.txt


In [ ]:
# --- Install rouge scorer ---
!pip install rouge-score --quiet

from rouge_score import rouge_scorer

# --- Load your human reference summary ---
with open("/content/EN2002a_summary.txt", "r") as f:
    reference_summary = f.read().strip()

# --- Load model-generated summaries (Whisper and Vosk transcripts) ---
with open("/content/EN2002a (2).txt", "r") as f:
    whisper_summary = f.read().strip()

with open("/content/EN2002a.txt", "r") as f:
    vosk_summary = f.read().strip()

# --- Initialize ROUGE scorer ---
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# --- Calculate ROUGE scores ---
print("ROUGE vs Whisper Transcript:")
scores_whisper = scorer.score(reference_summary, whisper_summary)
for metric, score in scores_whisper.items():
    print(f"{metric}: P={score.precision:.3f} R={score.recall:.3f} F1={score.fmeasure:.3f}")

print("\nROUGE vs Vosk Transcript:")
scores_vosk = scorer.score(reference_summary, vosk_summary)
for metric, score in scores_vosk.items():
    print(f"{metric}: P={score.precision:.3f} R={score.recall:.3f} F1={score.fmeasure:.3f}")


ROUGE vs Whisper Transcript:
rouge1: P=0.008 R=0.560 F1=0.016
rouge2: P=0.001 R=0.095 F1=0.003
rougeL: P=0.006 R=0.373 F1=0.011

ROUGE vs Vosk Transcript:
rouge1: P=0.008 R=0.600 F1=0.016
rouge2: P=0.001 R=0.108 F1=0.003
rougeL: P=0.005 R=0.387 F1=0.010


In [ ]:
!pip install transformers datasets rouge-score --quiet


In [ ]:
# --- Load transcripts ---
with open("/content/whisper_EN2002a.txt", "r") as f:
    whisper_text = f.read().strip()

with open("/content/vosk_EN2002a.txt", "r") as f:
    vosk_text = f.read().strip()

with open("/content/human_summary_EN2002a.txt", "r") as f:
    human_summary = f.read().strip()


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load T5 model
model_name = "t5-small"  # or "t5-base" if GPU memory allows
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate_summary(text, max_input=512, max_output=150):
    # Tokenize input
    inputs = tokenizer(
        "summarize: " + text,
        return_tensors="pt",
        max_length=max_input,
        truncation=True
    )
    # Generate summary
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_output,
        min_length=30,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

whisper_summary = generate_summary(whisper_text)
vosk_summary = generate_summary(vosk_text)

print("Whisper Summary:\n", whisper_summary)
print("\nVosk Summary:\n", vosk_summary)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Whisper Summary:
 we've got a browser, which comes up automatically with transcriptions. then when you go on the menu, you can select the summarisation box, which pops up, and an audio player. then when you go on the menu, you can select the summarisation box, which pops up, and then it loads up just the background window, so empty.

Vosk Summary:
 the here get certificate right so we really need to talk about the bodies are we just go around and everyone says what they've been doing that without anything live say yeah it sounds like you've done i could do that i've got a browser which comes up automatically with or transcription box and the topics and then when you go on the menu you can select some ization box which pops up and an audio player one am and as well say pop up a search and loads up just the background when nice empty.


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)

print("ROUGE vs Whisper Summary:")
print(scorer.score(human_summary, whisper_summary))

print("\nROUGE vs Vosk Summary:")
print(scorer.score(human_summary, vosk_summary))


ROUGE vs Whisper Summary:
{'rouge1': Score(precision=0.1724137931034483, recall=0.13333333333333333, fmeasure=0.15037593984962408), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.10344827586206896, recall=0.08, fmeasure=0.09022556390977444)}

ROUGE vs Vosk Summary:
{'rouge1': Score(precision=0.1414141414141414, recall=0.18666666666666668, fmeasure=0.16091954022988506), 'rouge2': Score(precision=0.01020408163265306, recall=0.013513513513513514, fmeasure=0.011627906976744186), 'rougeL': Score(precision=0.08080808080808081, recall=0.10666666666666667, fmeasure=0.09195402298850575)}


In [ ]:
# ROUGE scores
rouge_whisper = {
    'rouge1': {'P': 0.1724, 'R': 0.1333, 'F1': 0.1504},
    'rouge2': {'P': 0.0000, 'R': 0.0000, 'F1': 0.0000},
    'rougeL': {'P': 0.1034, 'R': 0.0800, 'F1': 0.0902},
}

rouge_vosk = {
    'rouge1': {'P': 0.1414, 'R': 0.1867, 'F1': 0.1609},
    'rouge2': {'P': 0.0102, 'R': 0.0135, 'F1': 0.0116},
    'rougeL': {'P': 0.0808, 'R': 0.1067, 'F1': 0.0920},
}

print("=== ROUGE Scores (Whisper) ===")
for k, v in rouge_whisper.items():
    print(f"{k.upper()}: Precision={v['P']:.4f}, Recall={v['R']:.4f}, F1={v['F1']:.4f}")

print("\n=== ROUGE Scores (Vosk) ===")
for k, v in rouge_vosk.items():
    print(f"{k.upper()}: Precision={v['P']:.4f}, Recall={v['R']:.4f}, F1={v['F1']:.4f}")


=== ROUGE Scores (Whisper) ===
ROUGE1: Precision=0.1724, Recall=0.1333, F1=0.1504
ROUGE2: Precision=0.0000, Recall=0.0000, F1=0.0000
ROUGEL: Precision=0.1034, Recall=0.0800, F1=0.0902

=== ROUGE Scores (Vosk) ===
ROUGE1: Precision=0.1414, Recall=0.1867, F1=0.1609
ROUGE2: Precision=0.0102, Recall=0.0135, F1=0.0116
ROUGEL: Precision=0.0808, Recall=0.1067, F1=0.0920


In [ ]:
# ROUGE scores
rouge_whisper = {
    'rouge1': {'P': 0.1724, 'R': 0.1333, 'F1': 0.1504},
    'rouge2': {'P': 0.0000, 'R': 0.0000, 'F1': 0.0000},
    'rougeL': {'P': 0.1034, 'R': 0.0800, 'F1': 0.0902},
}

rouge_vosk = {
    'rouge1': {'P': 0.1414, 'R': 0.1867, 'F1': 0.1609},
    'rouge2': {'P': 0.0102, 'R': 0.0135, 'F1': 0.0116},
    'rougeL': {'P': 0.0808, 'R': 0.1067, 'F1': 0.0920},
}

print("=== ROUGE Scores (Whisper) ===")
for k, v in rouge_whisper.items():
    print(f"{k.upper()}: Precision={v['P']:.4f}, Recall={v['R']:.4f}, F1={v['F1']:.4f}")

print("\n=== ROUGE Scores (Vosk) ===")
for k, v in rouge_vosk.items():
    print(f"{k.upper()}: Precision={v['P']:.4f}, Recall={v['R']:.4f}, F1={v['F1']:.4f}")

# Conclusion based on ROUGE-L F1 (or average F1)
if rouge_vosk['rougeL']['F1'] > rouge_whisper['rougeL']['F1']:
    print("\nConclusion: Vosk performed slightly better than Whisper based on ROUGE scores.")
elif rouge_vosk['rougeL']['F1'] < rouge_whisper['rougeL']['F1']:
    print("\nConclusion: Whisper performed slightly better than Vosk based on ROUGE scores.")
else:
    print("\nConclusion: Both models performed almost the same.")


=== ROUGE Scores (Whisper) ===
ROUGE1: Precision=0.1724, Recall=0.1333, F1=0.1504
ROUGE2: Precision=0.0000, Recall=0.0000, F1=0.0000
ROUGEL: Precision=0.1034, Recall=0.0800, F1=0.0902

=== ROUGE Scores (Vosk) ===
ROUGE1: Precision=0.1414, Recall=0.1867, F1=0.1609
ROUGE2: Precision=0.0102, Recall=0.0135, F1=0.0116
ROUGEL: Precision=0.0808, Recall=0.1067, F1=0.0920

Conclusion: Vosk performed slightly better than Whisper based on ROUGE scores.


In [ ]:
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)

print("ROUGE vs Whisper Summary:")
print(scorer.score(human_summary, whisper_summary))

print("\nROUGE vs Vosk Summary:")
print(scorer.score(human_summary, vosk_summary))


ROUGE vs Whisper Summary:
{'rouge1': Score(precision=0.1724137931034483, recall=0.13333333333333333, fmeasure=0.15037593984962408), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.10344827586206896, recall=0.08, fmeasure=0.09022556390977444)}

ROUGE vs Vosk Summary:
{'rouge1': Score(precision=0.1414141414141414, recall=0.18666666666666668, fmeasure=0.16091954022988506), 'rouge2': Score(precision=0.01020408163265306, recall=0.013513513513513514, fmeasure=0.011627906976744186), 'rougeL': Score(precision=0.08080808080808081, recall=0.10666666666666667, fmeasure=0.09195402298850575)}


In [ ]:
from pprint import pprint

# ROUGE results
rouge_whisper = {
    'rouge1': {'Precision': 0.1724, 'Recall': 0.1333, 'F1': 0.1504},
    'rouge2': {'Precision': 0.0000, 'Recall': 0.0000, 'F1': 0.0000},
    'rougeL': {'Precision': 0.1034, 'Recall': 0.0800, 'F1': 0.0902},
}

rouge_vosk = {
    'rouge1': {'Precision': 0.1414, 'Recall': 0.1867, 'F1': 0.1609},
    'rouge2': {'Precision': 0.0102, 'Recall': 0.0135, 'F1': 0.0116},
    'rougeL': {'Precision': 0.0808, 'Recall': 0.1067, 'F1': 0.0920},
}

print("=== ROUGE vs Whisper Summary ===")
pprint(rouge_whisper)

print("\n=== ROUGE vs Vosk Summary ===")
pprint(rouge_vosk)

# Conclusion
if rouge_vosk['rougeL']['F1'] > rouge_whisper['rougeL']['F1']:
    print("\nConclusion: Vosk performed slightly better than Whisper based on ROUGE-L F1 score.")
elif rouge_vosk['rougeL']['F1'] < rouge_whisper['rougeL']['F1']:
    print("\nConclusion: Whisper performed slightly better than Vosk based on ROUGE-L F1 score.")
else:
    print("\nConclusion: Both models performed almost the same based on ROUGE-L F1 score.")


=== ROUGE vs Whisper Summary ===
{'rouge1': {'F1': 0.1504, 'Precision': 0.1724, 'Recall': 0.1333},
 'rouge2': {'F1': 0.0, 'Precision': 0.0, 'Recall': 0.0},
 'rougeL': {'F1': 0.0902, 'Precision': 0.1034, 'Recall': 0.08}}

=== ROUGE vs Vosk Summary ===
{'rouge1': {'F1': 0.1609, 'Precision': 0.1414, 'Recall': 0.1867},
 'rouge2': {'F1': 0.0116, 'Precision': 0.0102, 'Recall': 0.0135},
 'rougeL': {'F1': 0.092, 'Precision': 0.0808, 'Recall': 0.1067}}

Conclusion: Vosk performed slightly better than Whisper based on ROUGE-L F1 score.


In [ ]:
!pip install transformers accelerate rouge-score sentencepiece


In [ ]:
from huggingface_hub import login
login()   # It will ask you to paste your token


In [ ]:
!pip install transformers accelerate rouge-score sentencepiece


In [ ]:
from huggingface_hub import login
login()   # It will ask you to paste your token


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", use_auth_token=True)


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
import librosa, librosa.display
import matplotlib.pyplot as plt
from pyannote.database.util import load_rttm
from pyannote.core import Segment

audio_path = "/content/EN2002a.wav"
y, sr = librosa.load(audio_path, sr=None)
duration = len(y)/sr

# load diag annotation
ann = load_rttm("/content/diarization_EN2002a_fixed.rttm")["EN2002a"]

plt.figure(figsize=(14,4))
librosa.display.waveshow(y, sr=sr)
ylim = plt.ylim()
for segment, track, label in ann.itertracks(yield_label=True):
    start = segment.start
    end = segment.end
    plt.axvspan(start, end, alpha=0.25, label=label)

plt.title("Waveform with diarization segments (overlay)")
plt.xlabel("Time (s)")
plt.tight_layout()
plt.savefig("/content/logs/diarization_waveform.png", dpi=150)
plt.close()
print("Saved waveform overlay to /content/logs/diarization_waveform.png")

# export RTTM head and logs
!head -n 40 /content/diarization_EN2002a_fixed.rttm > /content/logs/rttm_head.txt
!head -n 40 /content/diarized_transcript.txt > /content/logs/diarized_text_head.txt
!cp /content/summaries/t5_summary.txt /content/logs/
!cp /content/summaries/bart_summary.txt /content/logs/

print("Saved logs to /content/logs/. Zip it if needed:")
!zip -r -q /content/logs.zip /content/logs
print("Zipped logs: /content/logs.zip")


Saved waveform overlay to /content/logs/diarization_waveform.png
Saved logs to /content/logs/. Zip it if needed:
Zipped logs: /content/logs.zip


In [ ]:
!pip install graphviz
from graphviz import Digraph
g = Digraph(format='png')
g.attr(rankdir='LR', fontsize='12')
g.node('A','Audio (.wav)')
g.node('B','STT (Whisper / Vosk)')
g.node('C','Diarization (pyannote)')
g.node('D','Merge & RTTM')
g.node('E','Diarized Transcript')
g.node('F','Summarizer (T5/BART/LLaMA)')
g.node('G','Evaluation (DER / ROUGE)')

g.edges([('A','B'),('A','C'),('B','D'),('C','D'),('D','E'),('E','F'),('F','G')])
g.render('/content/architecture_diagram', cleanup=True)
print("Diagram saved to /content/architecture_diagram.png")


Diagram saved to /content/architecture_diagram.png
